# Hello pgvector: Create, store and query Ollama embeddings in PostgreSQL using pgvector

This notebook will teach you:
- How to create embeddings from content using a Local Ollama server.
- How to use PostgreSQL as a vector database and store embeddings data in it using pgvector.
- How to use embeddings retrieved from a vector database to augment LLM generation. 

We'll be using the example of creating a chatbot to answer questions about Timescale use cases, referencing content from the Timescale Developer Q+A blog posts. 

This is a great first step to building something like chatbot that can reference a company knowledge base or developer docs.

Let's get started!

Note: This notebook uses a PostgreSQL database with pgvector installed that's hosted on Timescale. You can create your own cloud PostgreSQL database in minutes [at this link](https://console.cloud.timescale.com/signup) to follow along. You can also use a local PostgreSQL database if you prefer.



### Configuration
- Setup Ollama 
    - `docker compose up -d`
    - `docker exec ollama ollama pull nomic-embed-text`
    - `docker exec ollama ollama pull llama3.2`
- Install Python
- Install and configure a python virtual environment. We recommend [Pyenv](https://github.com/pyenv/pyenv)
- Install the requirements for this notebook using the following command:

In [108]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [109]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import json
import tiktoken
import psycopg2
import ast
import pgvector
import math
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector


In [110]:
from dotenv import dotenv_values

load_dotenv()

OLLAMA_HOST = os.getenv("OLLAMA_HOST").strip("'\"")
DATABASE_URL = os.getenv("DATABASE_URL").strip("'\"")


## Part 1: Create Embeddings
First, we'll create embeddings using a Local Ollama server API on some text we want to augment our LLM with.
In this example, we'll use content from the Timescale blog about real world use cases.

In [111]:
# Load your CSV file into a pandas DataFrame
df = pd.read_csv('blog_posts_data.csv')
df.head()

,title,content,url
0,"How to Build a Weather Station With Elixir, Ne...",This is an installment of our “Community Membe...,https://www.timescale.com/blog/how-to-build-a-...
1,CloudQuery on Using PostgreSQL for Cloud Asset...,This is an installment of our “Community Membe...,https://www.timescale.com/blog/cloudquery-on-u...
2,How a Data Scientist Is Building a Time-Series...,This is an installment of our “Community Membe...,https://www.timescale.com/blog/how-a-data-scie...
3,How Conserv Safeguards History: Building an En...,This is an installment of our “Community Membe...,https://www.timescale.com/blog/how-conserv-saf...
4,How Messari Uses Data to Open the Cryptoeconom...,This is an installment of our “Community Membe...,https://www.timescale.com/blog/how-messari-use...


### 1.1 Calculate cost of embedding data
It's usually a good idea to calculate how much creating embeddings for your selected content will cost.
We use a number of helper functions to calculate a cost estimate before creating the embeddings to help us avoid surprises.

For this toy example, since we're using a small dataset, the total cost will be less than $0.01.

In [112]:
# Helper functions to help us create the embeddings

# Helper func: calculate number of tokens
def num_tokens_from_string(string: str, encoding_name = "cl100k_base") -> int:
    if not string:
        return 0
    # Returns the number of tokens in a text string
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Helper function: calculate length of essay
def get_essay_length(essay):
    word_list = essay.split()
    num_words = len(word_list)
    return num_words

# Helper function: calculate cost of embedding num_tokens
# Just to check how much money we are saving!
# Assumes we're using the text-embedding-ada-002 model
def get_embedding_cost(num_tokens):
    return num_tokens/1000*0.00002

# Helper function: calculate total cost of embedding all content in the dataframe
def get_total_embeddings_cost():
    total_tokens = 0
    for i in range(len(df.index)):
        text = df['content'][i]
        token_len = num_tokens_from_string(text)
        total_tokens = total_tokens + token_len
    total_cost = get_embedding_cost(total_tokens)
    return total_cost

In [113]:
# quick check on total token amount for price estimation
total_cost = get_total_embeddings_cost()
print("estimated price to embed this content = $" + str(total_cost))

estimated price to embed this content = $0.00120356


### 1.2 Create smaller chunks of content
A Local Ollama server API has a limit to the maximum amount of tokens it create create an embedding for in a single request. To get around this limit we'll break up our text into smaller chunks. In general its a best practice to create embeddings of a certain size in order to get better retrieval. For our purposes, we'll aim for chunks of around 512 tokens each.

Note: If you prefer to skip this step, you can use use the provided file: blog_data_and_embeddings.csv which contains the data and embeddings that you'll generate in this step.

In [114]:
# list for chunked content and embeddings
new_list = []
# Split up the text into token sizes of around 512 tokens
for i in range(len(df.index)):
    text = df['content'][i]
    token_len = num_tokens_from_string(text)
    if token_len <= 512:
        new_list.append([df['title'][i], df['content'][i], df['url'][i], token_len])
    else:
        # add content to the new list in chunks
        start = 0
        ideal_token_size = 512
        # 1 token ~ 3/4 of a word
        ideal_size = int(ideal_token_size // (4/3))
        end = ideal_size
        #split text by spaces into words
        words = text.split()

        #remove empty spaces
        words = [x for x in words if x != ' ']

        total_words = len(words)
        
        #calculate iterations
        chunks = total_words // ideal_size
        if total_words % ideal_size != 0:
            chunks += 1
        
        new_content = []
        for j in range(chunks):
            if end > total_words:
                end = total_words
            new_content = words[start:end]
            new_content_string = ' '.join(new_content)
            new_content_token_len = num_tokens_from_string(new_content_string)
            if new_content_token_len > 0:
                new_list.append([df['title'][i], new_content_string, df['url'][i], new_content_token_len])
            start += ideal_size
            end += ideal_size

In [115]:
# Helper function: get embeddings for a text
import requests


def get_embeddings(text):

    url = f"http://{OLLAMA_HOST}:11434/api/embeddings"
    
    payload = {
        "model": "nomic-embed-text",
        "prompt": text.replace("\n", " ")
    }
    
    headers = {
        "Content-Type": "application/json"
    }
    
    response = requests.post(url, data=json.dumps(payload), headers=headers)
    
    if response.status_code == 200:
        return response.json()["embedding"]
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")


In [116]:
# Create embeddings for each piece of content
for i in range(len(new_list)):
    text = new_list[i][1]
    embedding = get_embeddings(text)
    new_list[i].append(embedding)

# Create a new dataframe from the list
df_new = pd.DataFrame(new_list, columns=['title', 'content', 'url', 'tokens', 'embeddings'])
df_new.head()

ConnectionError: HTTPConnectionPool(host='192.168.51.12', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x713468ca1640>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [59]:
# Save the dataframe with embeddings as a CSV file
df_new.to_csv('blog_data_and_embeddings.csv', index=False)
# It may also be useful to save as a json file, but we won't use this in the tutorial
df_new.to_json('blog_data_and_embeddings.json') 

## Part 2: Store embeddings with pgvector
In this section, we'll store our embeddings and associated metadata. 

We'll use PostgreSQL as a vector database, with the pgvector extension. 

You can create a cloud PostgreSQL database for free on [Timescale](https://console.cloud.timescale.com/signup) or use a local PostgreSQL database for this step.

### 2.2 Connect to and configure your vector database


In [61]:
# Connect to PostgreSQL database in Timescale using connection string
conn = psycopg2.connect(DATABASE_URL)
cur = conn.cursor()

#install pgvector 
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
conn.commit()

#install pgvectorscale 
cur.execute("CREATE EXTENSION IF NOT EXISTS vectorscale CASCADE;")
conn.commit()

# Register the vector type with psycopg2
register_vector(conn)

# Create table to store embeddings and metadata
table_create_command = """
CREATE TABLE IF NOT EXISTS embeddings (
            id bigserial primary key, 
            title text,
            url text,
            content text,
            tokens integer,
            embedding vector(768)
            );
            """

cur.execute(table_create_command)
cur.close()
conn.commit()

Optional: Uncomment and execute the following code only if you need to read the embeddings and metadata from the provided CSV file

In [62]:
# Uncomment and execute this cell only if you need to read the blog data and embeddings from the provided CSV file
# Otherwise, skip to next cell
'''
df = pd.read_csv('blog_data_and_embeddings.csv')
titles = df['title']
urls = df['url']
contents = df['content']
tokens = df['tokens']
embeds = [list(map(float, ast.literal_eval(embed_str))) for embed_str in df['embeddings']]

df_new = pd.DataFrame({
    'title': titles,
    'url': urls,
    'content': contents,
    'tokens': tokens,
    'embeddings': embeds
})
'''

"\ndf = pd.read_csv('blog_data_and_embeddings.csv')\ntitles = df['title']\nurls = df['url']\ncontents = df['content']\ntokens = df['tokens']\nembeds = [list(map(float, ast.literal_eval(embed_str))) for embed_str in df['embeddings']]\n\ndf_new = pd.DataFrame({\n    'title': titles,\n    'url': urls,\n    'content': contents,\n    'tokens': tokens,\n    'embeddings': embeds\n})\n"

### 2.3 Ingest and store vector data into PostgreSQL using pgvector
In this section, we'll batch insert our embeddings and metadata into PostgreSQL and also create an index to help speed up search.

In [63]:
register_vector(conn)
cur = conn.cursor()

In [64]:
# Remind ourselves of the dataframe structure
df_new.head()

,title,content,url,tokens,embeddings
0,"How to Build a Weather Station With Elixir, Ne...",This is an installment of our “Community Membe...,https://www.timescale.com/blog/how-to-build-a-...,501,"[0.3103351891040802, 0.2868889272212982, -2.38..."
1,"How to Build a Weather Station With Elixir, Ne...",capture weather and environmental data. In all...,https://www.timescale.com/blog/how-to-build-a-...,512,"[0.4867981970310211, 1.2176804542541504, -3.53..."
2,"How to Build a Weather Station With Elixir, Ne...",command in their database migration:SELECT cre...,https://www.timescale.com/blog/how-to-build-a-...,374,"[0.6124968528747559, 1.4722315073013306, -3.97..."
3,CloudQuery on Using PostgreSQL for Cloud Asset...,This is an installment of our “Community Membe...,https://www.timescale.com/blog/cloudquery-on-u...,519,"[-0.25388479232788086, 1.5728431940078735, -1...."
4,CloudQuery on Using PostgreSQL for Cloud Asset...,Architecture with CloudQuery SDK- Writing plug...,https://www.timescale.com/blog/cloudquery-on-u...,511,"[1.7953698635101318, 2.0240869522094727, -2.54..."


Batch insert embeddings using psycopg2's ```execute_values()```

In [65]:
#Batch insert embeddings and metadata from dataframe into PostgreSQL database

# Prepare the list of tuples to insert
data_list = [(row['title'], row['url'], row['content'], int(row['tokens']), np.array(row['embeddings'])) for index, row in df_new.iterrows()]
# Use execute_values to perform batch insertion
execute_values(cur, "INSERT INTO embeddings (title, url, content, tokens, embedding) VALUES %s", data_list)
# Commit after we insert all embeddings
conn.commit()

Sanity check by running some simple queries against the embeddings table

In [66]:
cur.execute("SELECT COUNT(*) as cnt FROM embeddings;")
num_records = cur.fetchone()[0]
print("Number of vector records in table: ", num_records,"\n")
# Correct output should be 129

Number of vector records in table:  129 



In [67]:
# print the first record in the table, for sanity-checking
cur.execute("SELECT * FROM embeddings LIMIT 1;")
records = cur.fetchall()
print("First record in table: ", records)

First record in table:  [(1, 'How to Build a Weather Station With Elixir, Nerves, and TimescaleDB', 'https://www.timescale.com/blog/how-to-build-a-weather-station-with-elixir-nerves-and-timescaledb/', 'This is an installment of our “Community Member Spotlight” series, where we invite our customers to share their work, shining a light on their success and inspiring others with new ways to use technology to solve problems.In this edition,Alexander Koutmos, author of the Build a Weather Station with Elixir and Nerves book, joins us to share how he uses Grafana and TimescaleDB to store and visualize weather data collected from IoT sensors.About the teamThe bookBuild a Weather Station with Elixir and Nerveswas a joint effort between Bruce Tate, Frank Hunleth, and me.I have been writing software professionally for almost a decade and have been working primarily with Elixir since 2016. I currently maintain a few Elixir libraries onHexand also runStagira, a software consultancy company.Bruce T

Create index on embedding column for faster cosine similarity comparison

In [68]:
# Create an index on the data for faster retrieval
# this isn't really needed for 129 vectors, but it shows the usage for larger datasets
# Note: always create this type of index after you have data already inserted into the DB

# for different tuning suggestions check this: https://github.com/timescale/pgvectorscale?tab=readme-ov-file#tuning
cur.execute('CREATE INDEX embedding_idx ON embeddings USING diskann (embedding);')
conn.commit()

## Part 3: Nearest Neighbor Search using pgvector

In this final part of the tutorial, we will query our embeddings table. 

We'll showcase an example of RAG: Retrieval Augmented Generation, where we'll retrieve relevant data from our vector database and give it to the LLM as context to use when it generates a response to a prompt.

In [69]:
import requests
import json

def get_completion_from_messages(messages, model="llama3.2:latest", temperature=0, max_tokens=1000):
    # Ollama API endpoint
    url = f"http://{OLLAMA_HOST}:11434/api/generate"
    
    # Prepare the prompt
    prompt = ""
    for message in messages:
        role = message["role"]
        content = message["content"]
        prompt += f"{role.capitalize()}: {content}\n"
    
    # Prepare the request payload
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    
    # Send the request to Ollama
    response = requests.post(url, json=payload)
    
    if response.status_code == 200:
        result = json.loads(response.text)
        return result["response"]
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")


In [70]:
# Helper function: Get top 3 most similar documents from the database
def get_top3_similar_docs(query_embedding, conn):
    embedding_array = np.array(query_embedding)
    # Register pgvector extension
    register_vector(conn)
    cur = conn.cursor()
    # Get the top 3 most similar documents using the KNN <=> operator
    cur.execute("SELECT content FROM embeddings ORDER BY embedding <=> %s LIMIT 3", (embedding_array,))
    top3_docs = cur.fetchall()
    return top3_docs

### 3.1 Define a prompt for the LLM
Here we'll define the prompt we want the LLM to provide a reponse to.

We've picked an example relevant to the blog post data stored in the database.

In [71]:
# Question about Timescale we want the model to answer
input = "How is Timescale used in IoT?"

In [72]:
# Function to process input with retrieval of most similar documents from the database
def process_input_with_retrieval(user_input):
    delimiter = "```"

    #Step 1: Get documents related to the user input from database
    related_docs = get_top3_similar_docs(get_embeddings(user_input), conn)

    # Step 2: Get completion from Ollama API
    # Set system message to help set appropriate tone and context for model
    system_message = f"""
    You are a friendly chatbot. \
    You can answer questions about timescaledb, its features and its use cases. \
    You respond in a concise, technically credible tone. \
    """

    # Prepare messages to pass to model
    # We use a delimiter to help the model understand the where the user_input starts and ends
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"{delimiter}{user_input}{delimiter}"},
        {"role": "assistant", "content": f"Relevant Timescale case studies information: \n {related_docs[0][0]} \n {related_docs[1][0]} {related_docs[2][0]}"}   
    ]

    final_response = get_completion_from_messages(messages)
    return final_response

In [73]:
response = process_input_with_retrieval(input)
print(input)
print(response)

How is Timescale used in IoT?
TimescaleDB is particularly well-suited for IoT projects due to its ability to efficiently handle large amounts of time-stamped sensor data. By leveraging TimescaleDB's performance capabilities, you can reduce latency in queries and improve overall system responsiveness.

In the context of IoT, TimescaleDB can be used to store and analyze sensor data from various sources, such as temperature, humidity, pressure, or other environmental factors. The database's ability to compress and aggregate data helps keep table sizes in check, ensuring optimal performance even with growing data volumes.

Some potential use cases for TimescaleDB in IoT include:

1. Real-time monitoring of environmental conditions
2. Predictive maintenance for industrial equipment
3. Analyzing sensor data from smart cities or buildings

By utilizing TimescaleDB's features and capabilities, you can build efficient and scalable solutions that cater to the unique demands of IoT applications.


In [74]:
# We can also ask the model questions about specific documents in the database
input_2 = "Tell me about Edeva and Hopara. How do they use Timescale?"
response_2 = process_input_with_retrieval(input_2)
print(input_2)
print(response_2)

Tell me about Edeva and Hopara. How do they use Timescale?
Edeva and Hopara are two companies that have successfully implemented TimescaleDB to manage their time-series data. Here's a brief overview of how they use Timescale:

**Edeva**

Edeva uses TimescaleDB to store and analyze traffic data from their dynamic speed bumps, which contain hundreds of millions of records. They create a materialized view using continuous aggregations, which allows them to efficiently query and visualize their data.

The materialized view uses the `timescaledb_experimental.time_bucket_ng` function to bucket time-series data by month, and then applies aggregation functions to calculate percentiles of vehicle speed. This allows Edeva to quickly fetch data for graphs and maps in their analytics platform, EdevaLive.

**Hopara**

Hopara uses TimescaleDB as the backend database for their IoT project, which involves monitoring temperature and humidity sensors. They connect to TimescaleDB using PHP and Yii 2, and